In [1]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
np.random.seed(0)

In [2]:
file_path = 'modified_dataset/us_project_new_columns.csv'

us_ds = pd.read_csv(file_path, sep=',')

In [3]:
us_ds['period_begin'] = pd.to_datetime(us_ds['period_begin'])
us_ds.shape

(1789212, 68)

In [4]:
feature_names = us_ds.columns

# display the min max value of each feature
for feature in feature_names:
    print(f"{feature}: min={us_ds[feature].min()}, max={us_ds[feature].max()}")

period_begin: min=2012-01-01 00:00:00, max=2024-08-01 00:00:00
period_end: min=2012-01-31, max=2024-08-31
period_duration: min=30, max=30
region_type: min=place, max=place
region_type_id: min=6, max=6
table_id: min=3, max=38773
is_seasonally_adjusted: min=f, max=f
region: min=Abbeville, LA, max=Zumbrota, MN
city: min=Abbeville, max=Zumbrota
state: min=Alabama, max=Wisconsin
state_code: min=AK, max=WV
property_type: min=All Residential, max=Townhouse
property_type_id: min=-1, max=13
median_sale_price: min=625.0, max=37000000.0
median_sale_price_mom: min=-0.995427582797825, max=289.0
median_sale_price_yoy: min=-0.99525, max=88999.0
median_list_price: min=500.0, max=999999999.0
median_list_price_mom: min=-0.999436999999437, max=1597.7210215827338
median_list_price_yoy: min=-0.999375999999376, max=1820.49362295082
median_ppsf: min=0.285521898339928, max=925000.0
median_ppsf_mom: min=-0.9998472722829738, max=1567.9927875747485
median_ppsf_yoy: min=-0.9996949398412834, max=84975.65615141955


In [5]:
# compute z-score for numerical columns 
# z_scores = us_ds.select_dtypes(include=['float64', 'int64']).apply(zscore)

exclude_columns = ['period_duration', 'region_type_id', 'table_id', 'property_type_id', 
'parent_metro_region_metro_code', 'year', 'month', 'price_increased_mom', 
'price_increased_yoy', 'fast_selling']

# us_ds_ca = us_ds[us_ds['state_code'] == 'CA']
numeric_columns = us_ds.select_dtypes(include=['number']).columns

if exclude_columns:
    columns_for_outliers = [col for col in numeric_columns if col not in exclude_columns]
else:
    columns_for_outliers = numeric_columns

z_scores = us_ds[columns_for_outliers].apply(zscore)

# define a threshold for outliers
threshold = 5

# identify outliers
outliers = (z_scores.abs() > threshold)

# print summary of outliers
outlier_summary = outliers.sum().sort_values(ascending=False)
print("Outliers per feature:")
print(outlier_summary)

# get rows with any outliers
rows_with_outliers = us_ds[outliers.any(axis=1)]
print(f"Number of rows with outliers: {len(rows_with_outliers)}")
# rows_with_outliers.head(100)

Outliers per feature:
sales_to_new_listings_ratio    13448
months_of_supply_mom           11608
pending_sales_mom              11347
homes_sold_yoy                 11210
months_of_supply_yoy           11182
new_listings_mom               10329
months_of_supply               10320
homes_sold                     10102
homes_sold_mom                 10075
median_sale_price              10058
new_listings_yoy               10015
pending_sales                  10004
pending_sales_yoy               9954
new_listings                    9547
inventory_turnover              8584
avg_sale_to_list_mom            7986
inventory_yoy                   7739
inventory                       7592
avg_sale_to_list_yoy            6915
price_drops                     6662
inventory_mom                   6482
supply_demand_balance           5737
avg_sale_to_list                5605
median_dom_mom                  4702
median_dom_yoy                  4608
median_dom                      4583
price_drops_mom 

In [6]:
# replacing outliers
us_ds_copy = us_ds.copy()

for col in columns_for_outliers:
    # Calculate EMA for the column
    # ema = us_ds_ca_copy[col].ewm(span=10, adjust=False).mean()

    # calculate MA - moving average
    ma = us_ds_copy[col].rolling(window=2, center=True).mean()

    # Replace outliers with EMA
    # us_ds_ca_copy.loc[outliers[col], col] = ema[outliers[col]]
    us_ds_copy.loc[outliers[col], col] = ma[outliers[col]] 

# Step 4: Re-check rows with any outliers
z_scores_after = us_ds_copy[columns_for_outliers].apply(zscore)
outliers_after = z_scores_after.abs() > threshold
rows_with_outliers_after = outliers_after.any(axis=1).sum()

print(f"Number of rows with outliers after replacement: {rows_with_outliers_after}")

Number of rows with outliers after replacement: 141345


In [7]:
# display the min max value of each feature
for feature in feature_names:
    print(f"{feature}: min={us_ds_copy[feature].min()}, max={us_ds_copy[feature].max()}")

period_begin: min=2012-01-01 00:00:00, max=2024-08-01 00:00:00
period_end: min=2012-01-31, max=2024-08-31
period_duration: min=30, max=30
region_type: min=place, max=place
region_type_id: min=6, max=6
table_id: min=3, max=38773
is_seasonally_adjusted: min=f, max=f
region: min=Abbeville, LA, max=Zumbrota, MN
city: min=Abbeville, max=Zumbrota
state: min=Alabama, max=Wisconsin
state_code: min=AK, max=WV
property_type: min=All Residential, max=Townhouse
property_type_id: min=-1, max=13
median_sale_price: min=625.0, max=18732500.0
median_sale_price_mom: min=-0.995427582797825, max=144.44153137340936
median_sale_price_yoy: min=-0.99525, max=44499.53418803419
median_list_price: min=500.0, max=500136224.5
median_list_price_mom: min=-0.999436999999437, max=799.0551674920254
median_list_price_yoy: min=-0.999375999999376, max=910.2940284860416
median_ppsf: min=0.285521898339928, max=462584.8511772547
median_ppsf_mom: min=-0.9998472722829738, max=783.9859824447946
median_ppsf_yoy: min=-0.999694939

In [8]:
# # print summary of outliers
# outlier_summary_after = outliers_after.sum().sort_values(ascending=False)
# print("Outliers per feature:")
# print(outlier_summary_after)

In [9]:
# cleaned_data = us_ds[~outliers.any(axis=1)]
# print(f"Number of rows after removing outliers: {len(cleaned_data)}")

In [10]:
output_file = "modified_dataset/us_project_outliers.csv"

us_ds_copy.to_csv(output_file, index=False)